<a href="https://colab.research.google.com/github/vimesh630/Revenue_Forecasting/blob/main/Feature_Engineering_for_Revenue_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import os

# Define File Paths

In [ ]:
input_file_path = "/content/drive/MyDrive/VERGER/Revenue_Forecasting/preprocessed_forecast_data.csv"
output_dir = "/content/drive/MyDrive/VERGER/Revenue_Forecasting"
os.makedirs(output_dir, exist_ok=True)

# Load Dataset

In [ ]:
df = pd.read_csv(input_file_path)
df['date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2) + '-01')
df.sort_values(by='date', inplace=True)